<a href="https://colab.research.google.com/github/yeremiapane/Project-Capstone/blob/master/capstone_rumah123.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# data harga rumah di Jakarta, Bekasi, dan Depok dari sumber rumah123.com
df_jkt = pd.read_csv("/content/drive/MyDrive/data-collecting/01jakarta-rumah123.csv", header=0)
df_bgr = pd.read_csv("/content/drive/MyDrive/data-collecting/02bogor-rumah123.csv", header=0)
df_dpk = pd.read_csv("/content/drive/MyDrive/data-collecting/03depok-rumah123.csv", header=0)
df_tng = pd.read_csv("/content/drive/MyDrive/data-collecting/04tangerang-rumah123.csv", header=0)
df_tgs = pd.read_csv("/content/drive/MyDrive/data-collecting/04tangsel-rumah123.csv", header=0)
df_bks = pd.read_csv("/content/drive/MyDrive/data-collecting/05bekasi-rumah123.csv", header=0)

Mounted at /content/drive


In [ ]:
df_jkt

,ui-organisms-card-r123-featured__middle-section__price,ui-organisms-card-r123-featured__middle-section__address,attribute-text,attribute-text 2,attribute-text 3,attribute-value,attribute-value 2
0,"Rp 2,7 Miliar","Jelambar, Jakarta Barat",4.0,4.0,1.0,92 m²,218 m²
1,"Rp 2,7 Miliar","Jelambar, Jakarta Barat",4.0,3.0,NaN,92 m²,231 m²
2,Rp 35 Miliar,"Kemayoran, Jakarta Pusat",5.0,11.0,7.0,746 m²,1100 m²
3,"Rp 8,5 Miliar","Pantai Indah Kapuk, Jakarta Utara",4.0,3.0,2.0,300 m²,350 m²
4,"Rp 16,5 Miliar","Permata Buana, Jakarta Barat",3.0,3.0,1.0,300 m²,450 m²
...,...,...,...,...,...,...,...
1975,"Rp 10,5 Miliar","Kepa Duri, Jakarta Barat",5.0,8.0,1.0,234 m²,500 m²
1976,"Rp 15,5 Miliar","Pancoran, Jakarta Selatan",6.0,4.0,NaN,522 m²,450 m²
1977,"Rp 5,25 Miliar","Tomang, Jakarta Barat",4.0,2.0,NaN,228 m²,331 m²
1978,"Rp 4,5 Miliar","Cengkareng, Jakarta Barat",4.0,4.0,NaN,180 m²,135 m²


In [ ]:
df_jkt.isnull().sum()

ui-organisms-card-r123-featured__middle-section__price        8
ui-organisms-card-r123-featured__middle-section__address      8
attribute-text                                               88
attribute-text 2                                            103
attribute-text 3                                            794
attribute-value                                               1
attribute-value 2                                             9
dtype: int64

In [ ]:
''' data rumah Jakarta, Bekasi, Depok dari sumber rumah123.com'''
# # drop duplicates
# df_jkt.drop_duplicates()
# df_bgr.drop_duplicates()
# df_dpk.drop_duplicates()
# df_tng.drop_duplicates()
# df_tgs.drop_duplicates()
# df_bks.drop_duplicates()

# rename kolom
df_jkt = df_jkt.rename(columns={"ui-organisms-card-r123-featured__middle-section__price": "harga", "ui-organisms-card-r123-featured__middle-section__address": "lokasi", "attribute-text": "km-tidur", "attribute-text 2": "km-mandi", "attribute-value 2": "bangunan", "attribute-text 3": "garasi", "attribute-value": "tanah"})
df_bgr = df_bgr.rename(columns={"ui-organisms-card-r123-featured__middle-section__price": "harga", "ui-organisms-card-r123-featured__middle-section__address": "lokasi", "attribute-text": "km-tidur", "attribute-text 2": "km-mandi", "attribute-value 2": "bangunan", "attribute-text 3": "garasi", "attribute-value": "tanah"})
df_dpk = df_dpk.rename(columns={"ui-organisms-card-r123-featured__middle-section__price": "harga", "ui-organisms-card-r123-featured__middle-section__address": "lokasi", "attribute-text": "km-tidur", "attribute-text 2": "km-mandi", "attribute-value 2": "bangunan", "attribute-text 3": "garasi", "attribute-value": "tanah"})
df_tng = df_tng.rename(columns={"ui-organisms-card-r123-featured__middle-section__price": "harga", "ui-organisms-card-r123-featured__middle-section__address": "lokasi", "attribute-text": "km-tidur", "attribute-text 2": "km-mandi", "attribute-value 2": "bangunan", "attribute-text 3": "garasi", "attribute-value": "tanah"})
df_tgs = df_tgs.rename(columns={"ui-organisms-card-r123-featured__middle-section__price": "harga", "ui-organisms-card-r123-featured__middle-section__address": "lokasi", "attribute-text": "km-tidur", "attribute-text 2": "km-mandi", "attribute-value 2": "bangunan", "attribute-text 3": "garasi", "attribute-value": "tanah"})
df_bks = df_bks.rename(columns={"ui-organisms-card-r123-featured__middle-section__price": "harga", "ui-organisms-card-r123-featured__middle-section__address": "lokasi", "attribute-text": "km-tidur", "attribute-text 2": "km-mandi", "attribute-value 2": "bangunan", "attribute-text 3": "garasi", "attribute-value": "tanah"})

# menangani missing values pada garasi
df_jkt['garasi'].fillna(0, inplace=True)
df_bgr['garasi'].fillna(0, inplace=True)
df_dpk['garasi'].fillna(0, inplace=True)
df_tng['garasi'].fillna(0, inplace=True)
df_tgs['garasi'].fillna(0, inplace=True)
df_bks['garasi'].fillna(0, inplace=True)

# ubah value bangunan menjadi data numerik
df_jkt['bangunan'] = df_jkt.bangunan.str.extract('(\d+)')
df_bgr['bangunan'] = df_bgr.bangunan.str.extract('(\d+)')
df_dpk['bangunan'] = df_dpk.bangunan.str.extract('(\d+)')
df_tng['bangunan'] = df_tng.bangunan.str.extract('(\d+)')
df_tgs['bangunan'] = df_tgs.bangunan.str.extract('(\d+)')
df_bks['bangunan'] = df_bks.bangunan.str.extract('(\d+)')

# ubah value tanah menjadi data numerik
df_jkt['tanah'] = df_jkt.tanah.str.extract('(\d+)')
df_bgr['tanah'] = df_bgr.tanah.str.extract('(\d+)')
df_dpk['tanah'] = df_dpk.tanah.str.extract('(\d+)')
df_tng['tanah'] = df_tng.tanah.str.extract('(\d+)')
df_tgs['tanah'] = df_tgs.tanah.str.extract('(\d+)')
df_bks['tanah'] = df_bks.tanah.str.extract('(\d+)')

# menyamakan value lokasi
df_jkt['lokasi'] = "Jakarta"
df_bgr['lokasi'] = "Bogor"
df_dpk['lokasi'] = "Depok"
df_tng['lokasi'] = "Tangerang"
df_tgs['lokasi'] = "Tangerang Selatan"
df_bks['lokasi'] = "Bekasi"

# menghapus missing values
df_jkt.dropna(axis=0,inplace=True)
df_bgr.dropna(axis=0,inplace=True)
df_dpk.dropna(axis=0,inplace=True)
df_tng.dropna(axis=0,inplace=True)
df_tgs.dropna(axis=0,inplace=True)
df_bks.dropna(axis=0,inplace=True)

In [ ]:
# hilangkan tulisan 'Rp'
df_jkt['harga'] = df_jkt.harga.str.strip('Rp')
df_bgr['harga'] = df_bgr.harga.str.strip('Rp')
df_dpk['harga'] = df_dpk.harga.str.strip('Rp')
df_tng['harga'] = df_tng.harga.str.strip('Rp')
df_tgs['harga'] = df_tgs.harga.str.strip('Rp')
df_bks['harga'] = df_bks.harga.str.strip('Rp')

# reorder kolom
df_jkt = df_jkt.reindex(columns=['km-tidur', 'km-mandi', 'garasi', 'bangunan', 'tanah', 'lokasi', 'harga'])
df_bgr = df_bgr.reindex(columns=['km-tidur', 'km-mandi', 'garasi', 'bangunan', 'tanah', 'lokasi', 'harga'])
df_dpk = df_dpk.reindex(columns=['km-tidur', 'km-mandi', 'garasi', 'bangunan', 'tanah', 'lokasi', 'harga'])
df_tng = df_tng.reindex(columns=['km-tidur', 'km-mandi', 'garasi', 'bangunan', 'tanah', 'lokasi', 'harga'])
df_tgs = df_tgs.reindex(columns=['km-tidur', 'km-mandi', 'garasi', 'bangunan', 'tanah', 'lokasi', 'harga'])
df_bks = df_bks.reindex(columns=['km-tidur', 'km-mandi', 'garasi', 'bangunan', 'tanah', 'lokasi', 'harga'])

In [ ]:
# fungsi ubah string ke numerik
def converter(x):
  if 'juta' in x:
      return f"{(float(x.strip(' juta'))*1000000):,.2f}"
  elif 'Miliar' in x:
      return f"{(float(x.strip(' Miliar'))*1000000000):,.2f}"

# transformasi value harga
# jakarta
df_jkt['p0'] = df_jkt.harga.str.split("-", n=1, expand=True)[0]
df_jkt['p0'] = df_jkt['p0'].str.replace('.', '')
df_jkt['p0'] = df_jkt['p0'].str.replace(',', '.').apply(converter)

# bogor
df_bgr['p0'] = df_bgr.harga.str.split("-", n=1, expand=True)[0]
df_bgr['p0'] = df_bgr['p0'].str.replace('.', '')
df_bgr['p0'] = df_bgr['p0'].str.replace(',', '.').apply(converter)

# depok
df_dpk['p0'] = df_dpk.harga.str.split("-", n=1, expand=True)[0]
df_dpk['p0'] = df_dpk['p0'].str.replace('.', '')
df_dpk['p0'] = df_dpk['p0'].str.replace(',', '.').apply(converter)

# tangerang
df_tng['p0'] = df_tng.harga.str.split("-", n=1, expand=True)[0]
df_tng['p0'] = df_tng['p0'].str.replace('.', '')
df_tng['p0'] = df_tng['p0'].str.replace(',', '.').apply(converter)

# tangsel
df_tgs['p0'] = df_tgs.harga.str.split("-", n=1, expand=True)[0]
df_tgs['p0'] = df_tgs['p0'].str.replace('.', '')
df_tgs['p0'] = df_tgs['p0'].str.replace(',', '.').apply(converter)

# bekasi
df_bks['p0'] = df_bks.harga.str.split("-", n=1, expand=True)[0]
df_bks['p0'] = df_bks['p0'].str.replace('.', '')
df_bks['p0'] = df_bks['p0'].str.replace(',', '.').apply(converter)


<ipython-input-7-e2595ce0e822>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_jkt['p0'] = df_jkt['p0'].str.replace('.', '')
<ipython-input-7-e2595ce0e822>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_bgr['p0'] = df_bgr['p0'].str.replace('.', '')
<ipython-input-7-e2595ce0e822>:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_dpk['p0'] = df_dpk['p0'].str.replace('.', '')
<ipython-input-7-e2595ce0e822>:26: FutureWarning: The default value of regex will change from True to False in a future version. In addi

In [ ]:
# ganti value harga dengan p0
df_jkt = df_jkt.drop(columns='harga')
df_jkt = df_jkt.rename(columns={"p0": "harga"})

df_bgr = df_bgr.drop(columns='harga')
df_bgr = df_bgr.rename(columns={"p0": "harga"})

df_dpk = df_dpk.drop(columns='harga')
df_dpk = df_dpk.rename(columns={"p0": "harga"})

df_tng = df_tng.drop(columns='harga')
df_tng = df_tng.rename(columns={"p0": "harga"})

df_tgs = df_tgs.drop(columns='harga')
df_tgs = df_tgs.rename(columns={"p0": "harga"})

df_bks = df_bks.drop(columns='harga')
df_bks = df_bks.rename(columns={"p0": "harga"})


# hilangkan koma
df_jkt['harga'] = df_jkt['harga'].str.replace(',', '')
df_bgr['harga'] = df_bgr['harga'].str.replace(',', '')
df_dpk['harga'] = df_dpk['harga'].str.replace(',', '')
df_tng['harga'] = df_tng['harga'].str.replace(',', '')
df_tgs['harga'] = df_tgs['harga'].str.replace(',', '')
df_bks['harga'] = df_bks['harga'].str.replace(',', '')

In [ ]:
# ubah tipe data
df_jkt = df_jkt.astype({"bangunan":'float', "tanah": 'float', "km-tidur":'int', "km-mandi": 'int', "garasi": 'int', "harga": 'float'}) 
df_bgr = df_bgr.astype({"bangunan":'float', "tanah": 'float', "km-tidur":'int', "km-mandi": 'int', "garasi": 'int', "harga": 'float'}) 
df_dpk = df_dpk.astype({"bangunan":'float', "tanah": 'float', "km-tidur":'int', "km-mandi": 'int', "garasi": 'int', "harga": 'float'}) 
df_tng = df_tng.astype({"bangunan":'float', "tanah": 'float', "km-tidur":'int', "km-mandi": 'int', "garasi": 'int', "harga": 'float'}) 
df_tgs = df_tgs.astype({"bangunan":'float', "tanah": 'float', "km-tidur":'int', "km-mandi": 'int', "garasi": 'int', "harga": 'float'}) 
df_bks = df_bks.astype({"bangunan":'float', "tanah": 'float', "km-tidur":'int', "km-mandi": 'int', "garasi": 'int', "harga": 'float'}) 


In [ ]:
# menangani missing value pada harga
df_jkt['harga'].fillna(df_jkt['harga'].mean(), inplace=True)
df_bgr['harga'].fillna(df_bgr['harga'].mean(), inplace=True)
df_dpk['harga'].fillna(df_dpk['harga'].mean(), inplace=True)
df_tng['harga'].fillna(df_tng['harga'].mean(), inplace=True)
df_tgs['harga'].fillna(df_tgs['harga'].mean(), inplace=True)
df_bks['harga'].fillna(df_bks['harga'].mean(), inplace=True)



In [ ]:

# ganti tipe data price menjadi int
df_jkt['harga'] = df_jkt.harga.astype('int')
df_bgr['harga'] = df_bgr.harga.astype('int')
df_dpk['harga'] = df_dpk.harga.astype('int')
df_tng['harga'] = df_tng.harga.astype('int')
df_tgs['harga'] = df_tgs.harga.astype('int')
df_bks['harga'] = df_bks.harga.astype('int')


In [ ]:
df_jkt.describe()


,km-tidur,km-mandi,garasi,bangunan,tanah,harga
count,1862.000000,1862.000000,1862.000000,1862.000000,1862.000000,1.862000e+03
mean,4.494092,3.791622,1.372180,360.437164,312.852309,1.184371e+10
std,4.858416,3.844726,1.854324,573.805005,484.965611,2.103654e+10
min,1.000000,1.000000,0.000000,22.000000,16.000000,1.000000e+09
25%,3.000000,3.000000,0.000000,150.000000,102.000000,2.990000e+09
50%,4.000000,3.000000,1.000000,250.000000,180.000000,5.700000e+09
75%,5.000000,4.000000,2.000000,450.000000,362.750000,1.200000e+10
max,106.000000,57.000000,30.000000,20237.000000,9998.000000,2.700000e+11


In [ ]:
df_jkt.dtypes

km-tidur      int64
km-mandi      int64
garasi        int64
bangunan    float64
tanah       float64
lokasi       object
harga         int64
dtype: object

In [ ]:
df_jkt.isnull().sum()

km-tidur    0
km-mandi    0
garasi      0
bangunan    0
tanah       0
lokasi      0
harga       0
dtype: int64

In [ ]:
df_jkt

,km-tidur,km-mandi,garasi,bangunan,tanah,lokasi,harga
0,4,4,1,218.0,92.0,Jakarta,2700000000
1,4,3,0,231.0,92.0,Jakarta,2700000000
2,5,11,7,1100.0,746.0,Jakarta,35000000000
3,4,3,2,350.0,300.0,Jakarta,8500000000
4,3,3,1,450.0,300.0,Jakarta,16500000000
...,...,...,...,...,...,...,...
1975,5,8,1,500.0,234.0,Jakarta,10500000000
1976,6,4,0,450.0,522.0,Jakarta,15500000000
1977,4,2,0,331.0,228.0,Jakarta,5250000000
1978,4,4,0,135.0,180.0,Jakarta,4500000000


In [ ]:
# df_jkt.drop_duplicates() # 1700an
# drop duplicates
df_jkt.drop_duplicates(inplace=True)
df_bgr.drop_duplicates(inplace=True)
df_dpk.drop_duplicates(inplace=True)
df_tng.drop_duplicates(inplace=True)
df_tgs.drop_duplicates(inplace=True)
df_bks.drop_duplicates(inplace=True)

In [ ]:
df_jkt

,km-tidur,km-mandi,garasi,bangunan,tanah,lokasi,harga
0,4,4,1,218.0,92.0,Jakarta,2700000000
1,4,3,0,231.0,92.0,Jakarta,2700000000
2,5,11,7,1100.0,746.0,Jakarta,35000000000
3,4,3,2,350.0,300.0,Jakarta,8500000000
4,3,3,1,450.0,300.0,Jakarta,16500000000
...,...,...,...,...,...,...,...
1975,5,8,1,500.0,234.0,Jakarta,10500000000
1976,6,4,0,450.0,522.0,Jakarta,15500000000
1977,4,2,0,331.0,228.0,Jakarta,5250000000
1978,4,4,0,135.0,180.0,Jakarta,4500000000


In [ ]:
def outlier(dataframe):
  df = dataframe
  q1 = np.percentile(df['km-tidur'], 25, interpolation = 'midpoint')
  q3 = np.percentile(df['km-tidur'], 75, interpolation = 'midpoint')
  iqr = q3-q1
  # upper = np.where(df['km-tidur'] >= (q3 + 1.5*iqr))
  # lower = np.where(df['km-tidur'] <= (q1 - 1.5*iqr))
  upper = q3 + 1.5*iqr
  lower = q1 - 1.5*iqr
  df.drop(df[df['km-tidur'] >= upper].index, inplace=True)
  df.drop(df[df['km-tidur'] <= lower].index, inplace=True)
  # df.drop(upper[0], inplace=True)
  # df.drop(lower[0], inplace=True)
  return df

In [ ]:
df_jkt = outlier(df_jkt)
df_bgr = outlier(df_bgr)
df_dpk = outlier(df_dpk)
df_tng = outlier(df_tng)
df_tgs = outlier(df_tgs)
df_bks = outlier(df_bks)

In [ ]:
jkt = df_jkt[ (df_jkt['harga'] <= 120000000000 ) & (df_jkt['bangunan'] <= 3000) & (df_jkt['tanah'] <= 3000) ]
bgr = df_bgr[ (df_bgr['harga'] <= 120000000000 ) & (df_bgr['bangunan'] <= 3000) & (df_bgr['tanah'] <= 3000) ]
dpk = df_dpk[ (df_dpk['harga'] <= 120000000000 ) & (df_dpk['bangunan'] <= 3000) & (df_dpk['tanah'] <= 3000) ]
tng = df_tng[ (df_tng['harga'] <= 120000000000 ) & (df_tng['bangunan'] <= 3000) & (df_tng['tanah'] <= 3000) ]
tgs = df_tgs[ (df_tgs['harga'] <= 120000000000 ) & (df_tgs['bangunan'] <= 3000) & (df_tgs['tanah'] <= 3000) ]
bks = df_bks[ (df_bks['harga'] <= 120000000000 ) & (df_bks['bangunan'] <= 3000) & (df_bks['tanah'] <= 3000) ]

In [ ]:
jkt

,km-tidur,km-mandi,garasi,bangunan,tanah,lokasi,harga
0,4.0,4,1,218.0,92.0,Jakarta,2700000000
1,4.0,3,0,231.0,92.0,Jakarta,2700000000
2,5.0,11,7,1100.0,746.0,Jakarta,35000000000
3,4.0,3,2,350.0,300.0,Jakarta,8500000000
4,3.0,3,1,450.0,300.0,Jakarta,16500000000
...,...,...,...,...,...,...,...
1975,5.0,8,1,500.0,234.0,Jakarta,10500000000
1976,6.0,4,0,450.0,522.0,Jakarta,15500000000
1977,4.0,2,0,331.0,228.0,Jakarta,5250000000
1978,4.0,4,0,135.0,180.0,Jakarta,4500000000


In [ ]:
# # gabung ketiga dataframe menjadi 1
df = pd.concat([
    df_jkt.head(500),
    df_bgr.head(500),
    df_dpk.head(500),
    df_tng.head(500),
    df_tgs.head(500),
    df_bks.head(500),
    ])
df

,km-tidur,km-mandi,garasi,bangunan,tanah,lokasi,harga
0,4.0,4,1,218.0,92.0,Jakarta,2700000000
1,4.0,3,0,231.0,92.0,Jakarta,2700000000
2,5.0,11,7,1100.0,746.0,Jakarta,35000000000
3,4.0,3,2,350.0,300.0,Jakarta,8500000000
4,3.0,3,1,450.0,300.0,Jakarta,16500000000
...,...,...,...,...,...,...,...
859,2.0,1,1,70.0,112.0,Bekasi,2993723296
860,2.0,1,0,60.0,165.0,Bekasi,2993723296
862,3.0,2,2,63.0,54.0,Bekasi,2993723296
864,4.0,4,1,180.0,160.0,Bekasi,1900000000


In [ ]:
df_cp = df.reset_index(drop=True)
df_cp

,km-tidur,km-mandi,garasi,bangunan,tanah,lokasi,harga
0,4.0,4,1,218.0,92.0,Jakarta,2700000000
1,4.0,3,0,231.0,92.0,Jakarta,2700000000
2,5.0,11,7,1100.0,746.0,Jakarta,35000000000
3,4.0,3,2,350.0,300.0,Jakarta,8500000000
4,3.0,3,1,450.0,300.0,Jakarta,16500000000
...,...,...,...,...,...,...,...
2995,2.0,1,1,70.0,112.0,Bekasi,2993723296
2996,2.0,1,0,60.0,165.0,Bekasi,2993723296
2997,3.0,2,2,63.0,54.0,Bekasi,2993723296
2998,4.0,4,1,180.0,160.0,Bekasi,1900000000


In [ ]:
df_cp.to_csv('data_rumah123-new.csv',index=False)
!cp data_rumah123-new.csv "/content/drive/MyDrive/data-collecting"